In [1]:
#상태유지 스택 순환신경망
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tfprr
from tensorflow import keras
from tensorflow.keras import layers
tf.keras.backend.clear_session()

from keras.utils import np_utils

import pandas as pd
from sklearn import datasets
from keras.backend import tensorflow_backend as K
from keras import regularizers
import os
from keras import backend as K
import matplotlib.pyplot as plt
import glob


from pandas import DataFrame   
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, LSTM, Bidirectional, Dropout
from math import sqrt
from matplotlib import pyplot
from numpy import array
 
# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')
 
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
#	print("names: ",names)
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
#	print("names2: ", names)
	# put it all together
	agg = concat(cols, axis=1)
#	print("agg: ", agg)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
# create a differenced series / 데이터를 정지하게 하는 기능
def difference(dataset, interval=1):
	diff = list() #빈 리스트 생성 -> 간격만큼 뺀 값들, 작업하기 더 간단한 표현
	for i in range(interval, len(dataset)): #1부터 데이터셋 길이만큼 반복
		value = dataset[i] - dataset[i - interval]
		#print("diffenece()의 value: ", value, "/ dataset[i]-dataset[i-interval]: ",dataset[i]," / ", dataset[i - interval] )
		diff.append(value)
	return Series(diff)
 
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
	# extract raw values
	raw_values = series.values
	# transform data to be stationary
	diff_series = difference(raw_values, 1)
	diff_values = diff_series.values
	diff_values = diff_values.reshape(len(diff_values), 1)
    
	# rescale values to -1, 1 - 정규화 minmax
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
    #scaled_values: 차이만큼 뺴고 정규화한 값을 지도핛ㅂ
	# transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	# split into train and test sets
	train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	return scaler, train, test
 
# fit an LSTM network to training data
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
	# reshape training into [samples, timesteps, features]
	X, y = train[:, 0:n_lag], train[:, n_lag:]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	# design network
	model = Sequential()
	model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2])
                 , return_sequences=True,stateful=True, recurrent_dropout=0.2))
#	model.add(Dropout(0.15))
	model.add(LSTM(90, return_sequences=True, stateful=True, recurrent_dropout=0.2))
#	model.add(Dropout(0.15))
	model.add(Bidirectional(LSTM(60, return_sequences = False)))
	model.add(Dense(y.shape[1]))
    
	model.compile(loss='mse', optimizer='RMSprop')
	print(model.summary())
	# fit network
	for i in range(nb_epoch):
		print("Total epoch: %d / %d" % (i+1,nb_epoch))
		model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
		model.reset_states()
	return model
 
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
	# reshape input pattern to [samples, timesteps, features]
	X = X.reshape(1, 1, len(X))
	# make forecast
	forecast = model.predict(X, batch_size=n_batch)
	# convert to array
	return [x for x in forecast[0, :]]
 
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
	forecasts = list()
	for i in range(len(test)):
		X, y = test[i, 0:n_lag], test[i, n_lag:]
		# make forecast
		forecast = forecast_lstm(model, X, n_batch)
		# store the forecast
		forecasts.append(forecast)
	return forecasts
 
# invert differenced forecast
def inverse_difference(last_ob, forecast):
	# invert first forecast
	inverted = list()
	inverted.append(forecast[0] + last_ob)
	# propagate difference forecast using inverted first value
	for i in range(1, len(forecast)):
		inverted.append(forecast[i] + inverted[i-1])
	return inverted
 
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
	inverted = list()
	for i in range(len(forecasts)):
		# create array from forecast
		forecast = array(forecasts[i])
		forecast = forecast.reshape(1, len(forecast))
		# invert scaling
		inv_scale = scaler.inverse_transform(forecast)
		inv_scale = inv_scale[0, :]
		# invert differencing
		index = len(series) - n_test + i - 1
		last_ob = series.values[index]
		inv_diff = inverse_difference(last_ob, inv_scale)
		# store
		inverted.append(inv_diff)
	return inverted
 
# evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
	for i in range(n_seq):
		actual = [row[i] for row in test]
		predicted = [forecast[i] for forecast in forecasts]
		rmse = sqrt(mean_squared_error(actual, predicted))
		#print('t+%d RMSE: %f' % ((i+1), rmse))
 
# plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test, y):
	# plot the entire dataset in blue
	plt.figure(figsize=(25,5))
	plt.xticks(np.arange(0,series.shape[0],step=500),series.index, rotation=90)
	pyplot.plot(series.values)
	# plot the forecasts in red
	for i in range(len(forecasts)):
		off_s = len(series) - n_test + i - 1
		off_e = off_s + len(forecasts[i]) + 1
		xaxis = [x for x in range(off_s, off_e)]
		yaxis = [series.values[off_s]] + forecasts[i]
		plt.axhline(y=y, xmin=0.02, xmax=0.98, color='red')
		pyplot.plot(xaxis, yaxis, color='orange')
	# show the plot
pyplot.show()
 

NameError: name 'tf' is not defined

In [2]:
def read_file():
    files1 = glob.glob(os.path.join('../../', '*.csv'))
    #files = os.path.listdir('./')
    #train = pd.DataFrame()
    #xy = pd.DataFrame()

    li = []
    # print(len(files1))
    for file in range(len(files1)-1400):
        df = pd.read_csv(files1[file], error_bad_lines=False, header=0, index_col=False, usecols=[*range(0, 198)],
            names=["DATE","EA1","EA2","EA3","EA4","EA5","EA6","EA7","EA8","EA9","EA10","EA11","EA12","EA13","EA14","EA15","EA16","EA17","EA18","EA19","EA20","EA21","EA22","EA23","EA24","EA25","EA26","EA27","EA28","EA29","EA30","EA31","EA32","EA33","EA34","EA35","EA36","EA37","EA38","EA39","EA40","EA41","EA42","EA43","EA44","EA45","EA46","EA47","EA48","EA49","EA50","EA51","EA52","EA53","EA54","EA55","EA56","EA57","EA58","EA59","EA60","EA61","EA62","EA63","EA64","EA65","EA66","EA67","EA68","EA69","EA70","EA71","EA72","EA73","EA74","EA75","EA76","EA77","EA78","EA79","EA80","EA81","EA82","EA83","EA84","EA85","EA86","EA87","EA88","EA89","EA90","EA91","EA92","EA93","EA94","EA95","EA96","EA97","EA98","EA99","EA100","EA101","EA102","EA103","EA104","EA105","EA106","EA107","EA108","EA109","EA110","EA111","EA112","EA113","EA114","EA115","EA116","EA117","EA118","EA119","EA120","EA121","EA122","EA123","EA124","EA125","EA126","EA127","EA128","EA129","EA130","EA131","EA132","EA133","EA134","EA135","EA136","EA137","EA138","EA139","EA140","EA141","EA142","EA143","EA144","EA145","EA146","EA147","EA148","EA149","EA150","EA151","EA152","EA153","EA154","EA155","EA156","EA157","EA158","EA159","EA160","EA161","EA162","EA163","EA164","EA165","EA166","EA167","EA168","EA169","EA170","EA171","EA172","EA173","EA174","EA175","EA176","EA177","EA178","EA179","EA180","EA181","EA182","EA183","EA184","EA185","EA186","EA187","EA188","EA189","EA190","EA191","EA192","EA193","EA194","EA195","EA196","EA197","ED1","ED2","ED3","ED4","ED5","ED6","ED7","ED8","ED9","ED10","ED11","ED12","ED13","ED14","ED15","ED16","ED17","ED18","ED19","ED20","ED21","ED22","ED23","ED24","ED25","ED26","ED27","ED28","ED29","ED30","ED31","ED32","ED33","ED34","ED35","ED36","ED37","ED38","ED39","ED40","ED41","ED42","ED43","ED44","ED45","ED46","ED47","ED48","ED49","ED50","ED51","ED52","ED53","ED54","ED55","ED56","ED57","ED58","ED59","ED60","ED61","ED62","ED63","ED64","ED65","ED66","ED67","ED68","ED69","ED70","ED71","ED72","ED73","ED74","ED75","ED76","ED77","ED78","ED79","ED80","ED81","ED82","ED83","ED84","ED85","ED86","ED87","ED88","ED89","ED90","ED91","ED92","ED93","ED94","ED95","ED96","ED97","ED98","ED99","ED100","ED101","ED102","ED103","ED104","ED105","ED106","ED107","ED108","ED109","ED110","ED111","ED112","ED113","ED114","ED115","ED116","ED117","ED118","ED119","ED120","ED121","ED122","ED123","ED124","ED125","ED126","ED127","ED128","ED129","ED130","ED131","ED132","ED133","ED134","ED135","ED136","ED137","ED138","ED139","ED140","ED141","ED142","ED143","ED144","ED145","ED146","ED147","ED148","ED149","ED150","ED151","ED152","ED153","ED154","ED155","ED156","ED157","ED158","ED159","ED160","ED161","ED162","ED163","ED164","ED165","ED166","ED167","ED168","ED169","ED170","ED171","ED172","ED173","ED174","ED175","ED176","ED177","ED178","ED179","ED180","ED181","ED182","ED183","ED184","ED185","ED186","ED187","ED188","ED189","ED190","ED191","ED192","ED193","ED194","ED195","ED196","ED197","ED198","ED199","ED200","ED201","ED202","ED203","ED204","ED205","ED206","ED207","ED208","ED209","ED210","ED211","ED212","ED213","ED214","ED215","ED216","ED217","ED218","ED219","ED220","ED221","ED222","ED223","ED224","ED225","ED226","ED227","ED228","ED229","ED230","ED231","ED232","ED233","ED234","ED235","ED236","ED237","ED238","ED239","ED240","ED241","ED242","ED243","ED244","ED245","ED246","ED247","ED248","ED249","ED250","ED251","ED252","ED253","ED254","ED255","ED256","ED257","ED258","ED259","ED260","ED261","ED262","ED263","ED264","ED265","ED266","ED267","ED268","ED269","ED270","ED271","ED272","ED273","ED274","ED275","ED276","ED277","ED278","ED279","ED280","ED281","ED282","ED283","ED284","ED285","ED286","ED287","ED288","ED289","ED290","ED291","ED292","ED293","ED294","ED295","ED296","ED297","ED298","ED299","ED300","ED301","ED302","ED303","ED304","ED305","ED306","ED307","ED308","ED309","ED310","ED311","ED312","ED313","ED314","ED315","ED316","ED317","ED318","ED319","ED320","ED321","ED322","ED323","ED324","ED325","ED326","ED327","ED328","ED329","ED330","ED331","ED332","ED333","ED334","ED335","ED336","ED337","ED338","ED339","ED340","ED341","ED342","ED343","ED344","ED345","ED346","ED347","ED348","ED349","ED350","ED351","ED352","ED353","ED354","ED355","ED356","ED357","ED358","ED359","ED360","ED361","ED362","ED363","ED364","ED365","ED366","ED367","ED368","ED369","ED370","ED371","ED372","ED373","ED374","ED375","ED376","ED377","ED378","ED379","ED380","ED381","ED382","ED383","ED384","ED385","ED386","ED387","ED388","ED389","ED390","ED391","ED392","ED393","ED394","ED395","ED396","ED397","ED398","ED399","ED400","ED401","ED402","ED403","ED404","ED405","ED406","ED407","ED408","ED409","ED410","ED411","ED412","ED413","ED414","ED415","ED416","ED417","ED418","ED419","ED420","ED421","ED422","ED423","ED424","ED425","ED426","ED427","ED428","ED429","ED430","ED431","ED432","ED433","ED434","ED435","ED436","ED437","ED438","ED439","ED440","ED441","ED442","ED443","ED444","ED445","ED446","ED447","ED448","ED449","ED450","ED451","ED452","ED453","ED454","ED455","ED456","ED457","ED458","ED459","ED460","ED461","ED462","ED463","ED464","ED465","ED466","ED467","ED468","ED469","ED470","ED471","ED472","ED473","ED474","ED475","ED476","ED477","ED478","ED479","ED480","ED481","ED482","ED483","ED484","ED485","ED486","ED487","ED488","ED489","ED490","ED491","ED492","ED493","ED494","ED495","ED496","ED497","ED498","ED499","ED500","ED501","ED502","ED503","ED504","ED505","ED506","ED507","ED508","ED509","ED510","ED511","ED512","ED513","ED514","ED515","ED516","ED517","ED518","ED519","ED520","ED521","ED522","ED523","ED524","ED525","ED526","ED527","ED528","ED529","ED530","ED531","ED532","ED533","ED534","ED535","ED536","ED537","ED538","ED539","ED540","ED541","ED542","ED543","ED544","ED545","ED546","ED547","ED548","ED549","ED550","ED551","ED552","ED553","ED554","ED555","ED556","ED557","ED558","ED559","ED560","ED561","ED562","ED563","ED564","ED565","ED566","ED567","ED568","ED569","ED570","ED571","ED572","ED573","ED574","ED575","ED576","ED577","ED578","ED579","ED580","ED581","ED582","ED583","ED584","ED585","ED586","ED587","ED588","ED589","ED590","ED591","ED592","ED593","ED594","ED595","ED596","ED597","ED598","ED599","ED600","ED601","ED602","ED603","ED604","ED605","ED606","ED607","ED608","ED609","ED610","ED611","ED612","ED613","ED614","ED615","ED616","ED617","ED618","ED619","ED620","ED621","ED622","ED623","ED624","ED625","ED626","ED627","ED628","ED629","ED630","ED631","ED632","ED633","ED634","ED635","ED636","ED637","ED638","ED639","ED640","ED641","ED642","ED643","ED644","ED645","ED646","ED647","ED648","ED649","ED650","ED651","ED652","ED653","ED654","ED655","ED656","ED657","ED658","ED659","ED660","ED661","ED662","ED663","ED664","ED665","ED666","ED667","ED668","ED669","ED670","ED671","ED672","ED673","ED674","ED675","ED676","ED677","ED678","ED679","ED680","ED681","ED682","ED683","ED684","ED685","ED686","ED687","ED688","ED689","ED690","ED691","ED692","ED693","ED694","ED695","ED696","ED697","ED698","ED699","ED700","ED701","ED702","ED703","ED704","ED705","ED706","ED707","ED708","ED709","ED710","ED711","ED712","ED713","ED714","ED715","ED716","ED717","ED718","ED719","ED720","ED721","ED722","ED723","ED724","ED725","ED726","ED727","ED728","ED729","ED730","ED731","ED732","ED733","ED734","ED735","ED736","ED737","ED738","ED739","ED740","ED741","ED742","ED743","ED744","ED745","ED746","ED747","ED748","ED749","ED750","ED751","ED752","ED753","ED754","ED755","ED756","ED757","ED758","ED759","ED760","ED761","ED762","ED763","ED764","ED765","ED766","ED767","ED768","ED769","ED770","ED771","ED772","ED773","ED774","ED775","ED776","ED777","ED778","ED779","ED780","ED781","ED782","ED783","ED784","ED785","ED786","ED787","ED788","ED789","ED790","ED791","ED792","ED793","ED794","ED795","ED796","ED797","ED798","ED799","ED800","ED801","ED802","ED803","ED804","ED805","ED806","ED807","ED808","ED809","ED810","ED811","ED812","ED813","ED814","ED815","ED816","ED817","ED818","ED819","ED820","ED821","ED822","ED823","ED824","ED825","ED826","ED827","ED828","ED829","ED830","ED831","ED832","ED833","ED834","ED835","ED836","ED837","ED838","ED839","ED840","ED841","ED842","ED843","ED844","ED845","ED846","ED847","ED848","ED849","ED850","ED851","ED852","ED853","ED854","ED855","ED856","ED857","ED858","ED859","ED860","ED861","ED862","ED863","ED864","ED865","ED866","ED867","ED868","ED869","ED870","ED871","ED872","ED873","ED874","ED875","ED876","ED877","ED878","ED879","ED880","ED881","ED882","ED883","ED884","ED885","ED886","ED887","ED888","ED889","ED890","ED891","ED892","ED893","ED894","ED895","ED896","ED897","ED898","ED899","ED900","ED901","ED902","ED903","ED904","ED905","ED906","ED907","ED908","ED909","ED910","ED911","ED912","ED913","ED914","ED915","ED916","ED917","ED918","ED919","ED920","ED921","ED922","ED923","ED924","ED925","ED926","ED927","ED928","ED929","ED930","ED931","ED932","ED933","ED934","ED935","ED936","ED937","ED938","ED939","ED940","ED941","ED942","ED943","ED944","ED945","ED946","ED947","ED948","ED949","ED950","ED951","ED952","ED953","ED954","ED955","ED956","ED957","ED958","ED959","ED960","ED961","ED962","ED963","ED964","ED965","ED966","ED967","ED968","ED969","ED970","ED971","ED972","ED973","ED974","ED975","ED976","ED977","ED978","ED979","ED980","ED981","ED982","ED983","ED984","ED985","ED986","ED987","ED988","ED989","ED990","ED991","ED992","ED993","ED994","ED995","ED996","ED997","ED998","ED999","ED1000","ED1001","ED1002","ED1003","ED1004","ED1005","ED1006","ED1007","ED1008","ED1009","ED1010","ED1011","ED1012","ED1013","ED1014","ED1015","ED1016","ED1017","ED1018","ED1019","ED1020","ED1021","ED1022","ED1023","ED1024","ED1025","ED1026","ED1027","ED1028","ED1029","ED1030","ED1031","ED1032","ED1033","ED1034","ED1035","ED1036","ED1037","ED1038","ED1039","ED1040","ED1041","ED1042","ED1043","ED1044","ED1045","ED1046","ED1047","ED1048","ED1049","ED1050","ED1051","ED1052","ED1053","ED1054","ED1055","ED1056","ED1057","ED1058","ED1059","ED1060","ED1061","ED1062","ED1063","ED1064","ED1065","ED1066","ED1067","ED1068","ED1069","ED1070","ED1071","ED1072"]
            )   #오류나는 라인 생략, header=0: 1번째 행이 칼럼 이름

        #print(df.columns)
        #print(df.head()
        li.append(df)
    #    print(files1[file])
    #    else:
    #        test.append(xy)

    #print (li)
    xy = pd.concat(li, axis=0, ignore_index=True)
    return xy

In [3]:
    #Exh. Gas temp Cyl. A09 (TE25-A9)
def test_api():
        xy = read_file()
        #print(xy)
        #print(xy.shape)
        #xy.columns
        date = xy.iloc[27000:33000, 0:1].values
        value = xy.iloc[27000:33000, 89:90].values

        #print("컬럼명: ",xy.columns[89])

        date = date.ravel()
        date1 = date
        value = value.ravel()


        x1 = pd.DataFrame({xy.columns[89]: value},index=date)

        #value = std_based_outlier(value)
        #xy = MinMaxScaler(xy)
        #xy = pd.DataFrame(list(xy))

        #지수 이동평균 EMA (Exponetial Moving Average)
        origin_x =x1
        x1 = x1.ewm(170).mean()

        #print(df['EA90'].shape)
        #df['EA90']
        #df.columns

        # configure
        n_lag = 299 #모델링에 사용 된 지연 수
        n_seq = 300  #지속할 예측 단계의 수
        n_test = 1
        n_epochs = 10
        n_batch = 100
        n_neurons = 100

        # prepare data
        # 데이터를 변경하고 크기를 재조정한 다음, 지속성(persistence) 예제를 사용하여 감독된 학습 문제로 변환을 수행하고 테스트 세트를 훈련
        scaler, train, test = prepare_data(x1, n_test, n_lag, n_seq)

        # fit model
        model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

        #다른 배치사이즈 적용
        n_batch = 1
        X1, y1 = train[:, 0:n_lag], train[:, n_lag:]
        X1 = X1.reshape(X1.shape[0], 1, X1.shape[1])

        new_model =  Sequential()
        new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X1.shape[1], X1.shape[2])
                          ,return_sequences=True, stateful=True, dropout=0.2))
        new_model.add(LSTM(90, return_sequences=True,stateful=True, dropout=0.2))
        new_model.add(Bidirectional(LSTM(60, return_sequences = False)))
        new_model.add(Dense(y1.shape[1]))

        old_weights = model.get_weights()
        new_model.set_weights(old_weights)

        new_model.compile(loss='mse', optimizer='RMSprop')
        #print(new_model.summary())


        from keras.models import load_model
        new_model.save('model1.h5')
        # make forecasts
        forecasts = make_forecasts(new_model, n_batch, train, test, n_lag, n_seq)

        # inverse transform forecasts and test
        forecasts = inverse_transform(x1, forecasts, scaler, n_test+2)
        #print(forecasts)
        #print(len(forecasts[0]))
        actual = [row[n_lag:] for row in test]
        actual = inverse_transform(x1, actual, scaler, n_test+2)

        # evaluate forecasts
        evaluate_forecasts(actual, forecasts, n_lag, n_seq)

        # plot forecasts
        #plot_forecasts(x1, forecasts, n_test+2, 510.1)
        #x1


        # make forecasts
        forecasts = make_forecasts(new_model, n_batch, train, test, n_lag, n_seq)

        # inverse transform forecasts and test
        forecasts = inverse_transform(x1, forecasts, scaler, n_test+2)
        #print(forecasts)
        #print(len(forecasts[0]))
        actual = [row[n_lag:] for row in test]
        actual = inverse_transform(x1, actual, scaler, n_test+2)

        # evaluate forecasts
        evaluate_forecasts(actual, forecasts, n_lag, n_seq)

        # plot forecasts
        #plot_forecasts(x1, forecasts, n_test+2, 510.1)
        #x1

        a1 = []

        for i in range(len(forecasts[0])):
            a1.append(forecasts[0][i][0])   


        x1 = x1.to_numpy()
        x1 = x1.flatten().tolist()
    
        origin_x = origin_x.to_numpy()
        origin_x = origin_x.flatten().tolist()

        date1 = date1.tolist()
        date1 = date1[5000:]
        x1 = x1[5000:]

        return date1, x1, origin_x, a1

In [4]:
#date1, x1, origin_x, a1 = index()

In [5]:
#print(origin_x)